In [1]:
import numpy as np
import pandas as pd


credits = pd.read_csv(r"Data set/tmdb_5000_credits.csv")
movie = pd.read_csv(r"Data set/tmdb_5000_movies.csv")

In [2]:
movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [3]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [4]:
movies = movie.merge(credits, on='title')
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
#Important column to consider for movie recommendation system
#id,title
#1. genres
#2. keywords
#3. overview
#4. cast[Top 3]
#5. crew[Director]
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
#final data should have three colums
#1. movie_id
#2. title
#3. tags
#tags = overview + genres + keywords + cast + crew 

#Checking for null values
movies.isnull().sum()
#Dropping null values as only 3 values of data is null
movies.dropna(inplace=True)
#movies.duplicated().sum()
#No duplicates found

In [7]:
import ast

# Function to convert stringified JSON into a list of names
def convert(obj):
    L = []
    for dic in ast.literal_eval(obj):  # Parse the stringified JSON
        L.append(dic['name'].replace(" ", ""))  # Remove spaces from names
    return L

def convert_cast(obj):
    L = []
    counter = 0
    for dic in ast.literal_eval(obj):  # Parse the stringified JSON
        if counter != 3:  # Limit to top 3 cast members
            L.append(dic['name'].replace(" ", ""))
            counter += 1
        else:
            break
    return L

def convert_crew(obj):
    L=[]
    for dic in ast.literal_eval(obj):
        if dic['job']=='Director':
            L.append(dic['name'].replace(" ", ""))
            break
    return L

# Apply the function to each row in the 'genres' column
movies['genres'] = movies['genres'].apply(convert)
# Example: Join the genres into a single string (if needed)
movies['genres'] = movies['genres'].apply(lambda x: " ".join(x))
#print(movies['genres'])

movies['keywords']=movies['keywords'].apply(convert)
movies['keywords'] = movies['keywords'].apply(lambda x: " ".join(x))
#print(movies['keywords'])

movies['cast']= movies['cast'].apply(convert_cast)
movies['cast'] = movies['cast'].apply(lambda x: " ".join(x))
#print(movies['cast'])

movies['crew'] = movies['crew'].apply(convert_crew)
movies['crew'] = movies['crew'].apply(lambda x: " ".join(x))
#print(movies['crew'])


In [8]:
#Combining all the columns into a single column 'tags'
movies['tags'] = movies['overview'] + " " + movies['genres'] + " " + movies['keywords'] + " " + movies['cast'] + " " + movies['crew']
movies['tags'] = movies['tags'].apply(lambda x: x.lower())  # Convert to lowercase
movies = movies[['movie_id', 'title', 'tags']]
movies.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [9]:
# Save the final DataFrame to a CSV file in the 'Data set' folder
#movies.to_csv(r"Data set/final_movies_data.csv", index=False)
#vectorization of data
#coverting every tags into a vector using count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vector=cv.fit_transform(movies['tags']).toarray()
vector
cv.get_feature_names_out()
#if we see the output for the 5000 words it is considering action and actions as two different words but we want to consider them as one word so we will use stemming and lemmatization to convert them into a single word

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import nltk
from nltk.stem import PorterStemmer

ps=PorterStemmer()
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

movies['tags']=movies['tags'].apply(stem)
movies['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [ ]:
vector=cv.fit_transform(movies['tags']).toarray()
#we will calculate cosine angle to find the similarity between the movies 
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)
similarity

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]], shape=(4806, 4806))

In [ ]:
#saving the similarity matrix to a file for later use
# Save the similarity array to a .npy file
np.save(r"Data set/similarity.npy", similarity)

# Load the similarity array later
similarity = np.load(r"Data set/similarity.npy")


#now we will create a function to get the name of the movie based on the index of the movie
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]#tuples of (index,distance) of 5 most similar movies
 
    recommended_movies = []
    for i in movies_list:
        recommended_movies.append(movies.iloc[i[0]].title)# get the title of the movie using the index
    return recommended_movies